In [1]:
# enigma_machine.py

class EnigmaMachine:
    def __init__(self, rotors, reflector, plugboard_settings, initial_positions, ring_settings="AAA"):
        self.alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.rotors = []
        self.reflector = reflector
        self.plugboard = self._create_plugboard(plugboard_settings)
        self.ring_settings_indices = [self.alphabet.index(r) for r in ring_settings.upper()]

        # Define rotor wirings and notch positions
        # (Rotor_wiring, Notch_character)
        # These are examples; real Enigma had various rotor sets
        rotor_definitions = {
            "I": ("EKMFLGDQVZNTOWYHXUSPAIBRCJ", "Q"),
            "II": ("AJDKSIRUXBLHWTMCQGZNPYFVOE", "E"),
            "III": ("BDFHJLCPRTXVZNYEIWGAKMUSQO", "V"),
            "IV": ("ESOVPZJAYQUIRHXLNFTGKDCMWB", "J"),
            "V": ("VZBRGITYUPSDNHLXAWMJQOFECK", "Z")
        }

        # Define reflector wirings (example: UKW-B)
        reflector_definitions = {
            "B": "YRUHQSLDPXNGOKMIEBFZCWVJAT",
            "C": "FVPJIAOYEDRZXWGCTKUQSBNMHL"
        }

        self.reflector_wiring = reflector_definitions.get(reflector.upper(), reflector_definitions["B"])

        for i, rotor_name in enumerate(rotors):
            if rotor_name.upper() not in rotor_definitions:
                raise ValueError(f"Rotor {rotor_name} not defined.")
            wiring, notch = rotor_definitions[rotor_name.upper()]
            position = self.alphabet.index(initial_positions[i].upper())
            ring_setting_index = self.ring_settings_indices[i]
            self.rotors.append(self._create_rotor(wiring, notch, position, ring_setting_index))

    def _create_plugboard(self, settings):
        plugboard = {char: char for char in self.alphabet}
        if settings:
            pairs = settings.upper().split()
            for pair in pairs:
                if len(pair) == 2:
                    a, b = pair[0], pair[1]
                    if a in plugboard and b in plugboard and plugboard[a] == a and plugboard[b] == b:
                        plugboard[a] = b
                        plugboard[b] = a
                    else:
                        raise ValueError(f"Invalid plugboard setting: {pair}. Letters might be duplicated or not in alphabet.")
                else:
                    raise ValueError(f"Invalid plugboard pair: {pair}. Pairs must be two characters.")
        return plugboard

    def _create_rotor(self, wiring, notch_char, initial_position, ring_setting_index):
        return {
            "wiring": wiring,
            "position": initial_position, # Current visible letter's index
            "notch": self.alphabet.index(notch_char),
            "ring_setting": ring_setting_index # Ring setting relative to 'A'
        }

    def _rotor_step(self):
        # Implements rotor stepping mechanism (simplified)
        # Rightmost rotor steps on every key press
        # Middle rotor steps if rightmost rotor is at its notch
        # Leftmost rotor steps if middle rotor is at its notch AND rightmost rotor also just stepped (double stepping)

        # German Enigma stepping: The rotor to the left of a stepping rotor *also* steps
        # if it itself is at its notch position. This is the "double step" anomaly.

        rotor1 = self.rotors[2] # Rightmost
        rotor2 = self.rotors[1] # Middle
        rotor3 = self.rotors[0] # Leftmost

        # Check for double step condition
        step_rotor2 = rotor1["position"] == rotor1["notch"]
        step_rotor3 = rotor2["position"] == rotor2["notch"] and step_rotor2 # Rotor 2 must have been about to step

        # Step rightmost rotor always
        rotor1["position"] = (rotor1["position"] + 1) % 26

        if step_rotor2:
            rotor2["position"] = (rotor2["position"] + 1) % 26
            if step_rotor3: # If rotor2 stepped and was at its notch, rotor3 steps
                 rotor3["position"] = (rotor3["position"] + 1) % 26
        # Special case for middle rotor notch: if middle rotor is at notch, it steps the left rotor
        # This is independent of the right rotor stepping for the *next* cycle.
        elif rotor2["position"] == rotor2["notch"]: # Note: This covers a different aspect of Enigma stepping
             rotor3["position"] = (rotor3["position"] + 1) % 26


    def _pass_through_rotor(self, char_index, rotor, forward=True, is_first_pass=True):
        # Account for rotor position and ring setting
        effective_position = (rotor["position"] - rotor["ring_setting"] + 26) % 26
        char_index = (char_index + effective_position) % 26

        if forward:
            output_char = rotor["wiring"][char_index]
            output_index = self.alphabet.index(output_char)
        else: # Backward pass
            input_char = self.alphabet[char_index]
            output_index = rotor["wiring"].find(input_char)
            if output_index == -1:
                raise ValueError("Character not found in reverse rotor wiring - this shouldn't happen.")

        # Undo the shift from rotor position and ring setting for the output
        output_index = (output_index - effective_position + 26) % 26
        return output_index

    def process_char(self, char):
        char_upper = char.upper()
        if char_upper not in self.alphabet:
            return char # Pass non-alphabetic characters through unchanged

        # 1. Step rotors (before encryption/decryption of the character)
        self._rotor_step()

        # 2. Plugboard - Entry
        char_upper = self.plugboard[char_upper]
        current_index = self.alphabet.index(char_upper)

        # 3. Forward through rotors (right to left: Rotor III, II, I for a 3-rotor machine)
        for i in range(len(self.rotors) - 1, -1, -1):
            current_index = self._pass_through_rotor(current_index, self.rotors[i], forward=True)

        # 4. Reflector
        reflected_char = self.reflector_wiring[current_index]
        current_index = self.alphabet.index(reflected_char)

        # 5. Backward through rotors (left to right: Rotor I, II, III)
        for i in range(len(self.rotors)):
            current_index = self._pass_through_rotor(current_index, self.rotors[i], forward=False)

        # 6. Plugboard - Exit
        final_char = self.alphabet[current_index]
        final_char = self.plugboard[final_char]

        return final_char

    def process_text(self, text):
        processed_text = ""
        for char in text:
            processed_text += self.process_char(char)
        return processed_text

# --- Helper functions for file operations ---

def read_file(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error reading file {filepath}: {e}")
        return None

def write_file(filepath, content):
    try:
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"Output written to {filepath}")
    except Exception as e:
        print(f"Error writing file {filepath}: {e}")

# --- Main execution ---

def encrypt_file(input_filepath, output_filepath, rotor_choices, reflector_choice, plugboard_str, initial_rotor_positions, ring_settings_str):
    """Encrypts a file using the Enigma machine."""
    print("--- Enigma Encryption ---")
    print(f"Input file: {input_filepath}")
    print(f"Output file: {output_filepath}")
    print(f"Rotors (L-M-R): {' '.join(rotor_choices)}")
    print(f"Reflector: {reflector_choice}")
    print(f"Plugboard: '{plugboard_str}'")
    print(f"Initial Positions: {initial_rotor_positions}")
    print(f"Ring Settings: {ring_settings_str}")

    machine = EnigmaMachine(
        rotors=rotor_choices,    # e.g., ["I", "II", "III"] (Left, Middle, Right)
        reflector=reflector_choice,  # e.g., "B"
        plugboard_settings=plugboard_str, # e.g., "AB CD EF"
        initial_positions=initial_rotor_positions, # e.g., "A A A"
        ring_settings=ring_settings_str # e.g., "A A A" or "01 01 01"
    )

    plaintext = read_file(input_filepath)
    if plaintext is not None:
        ciphertext = machine.process_text(plaintext)
        write_file(output_filepath, ciphertext)
        print("Encryption complete.")

def decrypt_file(input_filepath, output_filepath, rotor_choices, reflector_choice, plugboard_str, initial_rotor_positions, ring_settings_str):
    """Decrypts a file using the Enigma machine. Uses the same settings as encryption."""
    print("\n--- Enigma Decryption ---")
    print(f"Input file: {input_filepath}")
    print(f"Output file: {output_filepath}")
    print(f"Rotors (L-M-R): {' '.join(rotor_choices)}")
    print(f"Reflector: {reflector_choice}")
    print(f"Plugboard: '{plugboard_str}'")
    print(f"Initial Positions: {initial_rotor_positions}")
    print(f"Ring Settings: {ring_settings_str}")

    # For decryption, the Enigma machine is set up exactly the same way as for encryption.
    # The reciprocal nature of the Enigma (largely due to the reflector and plugboard)
    # means that processing the ciphertext with the same settings will yield the plaintext.
    machine = EnigmaMachine(
        rotors=rotor_choices,
        reflector=reflector_choice,
        plugboard_settings=plugboard_str,
        initial_positions=initial_rotor_positions,
        ring_settings=ring_settings_str
    )

    ciphertext = read_file(input_filepath)
    if ciphertext is not None:
        plaintext = machine.process_text(ciphertext)
        write_file(output_filepath, plaintext)
        print("Decryption complete.")


if __name__ == "__main__":
    # --- Configuration ---
    # These settings MUST be the same for encryption and decryption.
    CHOSEN_ROTORS = ["I", "II", "III"]  # Order: Left, Middle, Right
    CHOSEN_REFLECTOR = "B"             # e.g., "B" or "C"
    PLUGBOARD_SETTINGS = "AZ BY CX"    # Pairs of letters to swap, e.g., "AB CD EF" or empty for none
    INITIAL_ROTOR_POSITIONS = "AAA"    # Starting letter for each rotor (Left, Middle, Right)
    RING_SETTINGS = "AAA"              # Ring settings (Grundstellung) for each rotor (L, M, R)

    # File paths
    PLAINTEXT_FILE = "plaintext.txt"
    ENCRYPTED_FILE = "encrypted.txt"
    DECRYPTED_FILE = "decrypted.txt"

    # --- Create a sample plaintext file ---
    sample_text = "HELLO WORLD THIS IS A TEST OF THE ENIGMA MACHINE PYTHON SCRIPT END"
    # sample_text = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA" # Good for testing stepping
    write_file(PLAINTEXT_FILE, sample_text.upper()) # Enigma typically used uppercase

    # --- Encrypt the file ---
    encrypt_file(
        PLAINTEXT_FILE,
        ENCRYPTED_FILE,
        CHOSEN_ROTORS,
        CHOSEN_REFLECTOR,
        PLUGBOARD_SETTINGS,
        INITIAL_ROTOR_POSITIONS,
        RING_SETTINGS
    )

    # --- Decrypt the file ---
    # Note: For decryption, you use the *exact same initial settings* as encryption.
    # The Enigma machine is reciprocal.
    decrypt_file(
        ENCRYPTED_FILE,
        DECRYPTED_FILE,
        CHOSEN_ROTORS,
        CHOSEN_REFLECTOR,
        PLUGBOARD_SETTINGS,
        INITIAL_ROTOR_POSITIONS,
        RING_SETTINGS
    )

    # --- Verify ---
    original_text = read_file(PLAINTEXT_FILE)
    decrypted_text = read_file(DECRYPTED_FILE)

    if original_text is not None and decrypted_text is not None:
        if original_text.strip().upper() == decrypted_text.strip().upper():
            print("\nVerification successful: Original and decrypted texts match!")
        else:
            print("\nVerification FAILED: Original and decrypted texts DO NOT match.")
            print(f"Original:\n'{original_text.strip().upper()}'")
            print(f"Decrypted:\n'{decrypted_text.strip().upper()}'")

Output written to plaintext.txt
--- Enigma Encryption ---
Input file: plaintext.txt
Output file: encrypted.txt
Rotors (L-M-R): I II III
Reflector: B
Plugboard: 'AZ BY CX'
Initial Positions: AAA
Ring Settings: AAA
Output written to encrypted.txt
Encryption complete.

--- Enigma Decryption ---
Input file: encrypted.txt
Output file: decrypted.txt
Rotors (L-M-R): I II III
Reflector: B
Plugboard: 'AZ BY CX'
Initial Positions: AAA
Ring Settings: AAA
Output written to decrypted.txt
Decryption complete.

Verification successful: Original and decrypted texts match!
